# Guided dFBA with uptake constraints from NMR

@author: Aidan Pavao for Massachusetts Host-Microbiome Center
 - Compute dynamic flux balance analysis (dFBA) solutions
 - Use python 3.8+ for best results
 - See README.md for dependencies
 
Copyright 2021 Massachusetts Host-Microbiome Center

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

   [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

## Import packages

In [3]:
import csv
import math

import cobra as cb
from cobra.flux_analysis import flux_variability_analysis as fva
from cobra.flux_analysis.loopless import loopless_solution as lsol
from cobra.flux_analysis import pfba
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors
import openpyxl as xl

## Conversion factors

Conversion factors to transform signal to concentration.

These were determined by as described in the Methods section of our C.diff HRMAS pre-print. _These are estimates of concentration and should not be taken as precise values._

In [4]:
# Scale Factors - calculated from HMDB reference spectra

scale_f = {
    'glc' : 1.,
    'ac'  : 1.,
    'alaL': 1.,
    'eto' : 1.,
    'lacS': 1.,
    'but' : 1.,
    'buta': 1.,
    'proL': 1.,
    '5apn': 1.,
    'leuL': 1.,
    'ival': 1.,
    'isocap': 1.,
    'thrL'  : 1.,
    '2obut' : 1.,
    '2hbut' : 1.,
    '2abut' : 1.,
    'ppa'   : 1.,
    'ppoh'  : 1.,
    'but2' : 1.,
}

pool = ['2obut', '2abut', 'ppa'] # pooling metabolites with product logistic fit
thr_mets = ['thrL', '2obut', '2abut', 'ppa', '2hbut', 'ppoh']

## Some key functions

Helper functions:
1. Evaluate logistic function at timepoint.
1. Evaluate derivative of logistic function at timepoint.
1. Update the uptake/secretion reaction bounds with the logistic derivative. This function evaluates both the logistic curve and its derivative at the timepoint, updates the bounds, and returns both solutions. Pass in the logistic coefficients calculated with MATLAB.


In [5]:
def logistic(x, *argv):
    """Evaluate the (sum of) logistic function at point x."""
    L = argv[0]
    k = argv[1]
    x0 = argv[2]
    sol = L/(1. + math.exp(-1*k*(x - x0)))
    if len(argv) == 3:
        return sol # typical solution
    elif len(argv) == 4:
        return sol + argv[3] # + C
    elif len(argv) == 5:
        return sol - logistic(x, L, *argv[3:]) # L2 = -L1
    else:
        return sol + logistic(x, *argv[3:])  # multiple curves

def ddx_logistic(x, *argv):
    """Evaluate the derivative of the (sum of) logistic function at point x."""
    L = argv[0]
    k = argv[1]
    x0 = argv[2]
    sol = k*L*math.exp(-k*(x - x0))/(1. + math.exp(-1*k*(x - x0)))**2
    if len(argv) <= 4:
        return sol  # typical solution
    elif len(argv) == 5:
        return sol - ddx_logistic(x, L, *argv[3:]) # L2 = -L1
    else:
        return sol + ddx_logistic(x, *argv[3:])  # multiple curves

def find_minmax_bounds(met, x, bounds):
    """An asinine and wasteful way to estimate minimum/maximum bounds of logistic using 95% confidence bounds for coefficients."""
    combinations = list(itertools.product(*bounds))
    for coeffset in combinations:
        all_bounds.append(ddx_logistic(x, *coeffset))
        all_signal.append(logistic(x, *coeffset))
    return min(all_bounds), max(all_bounds), min(all_signal), max(all_signal)

def thr_time_transform(t):
    y = [0, 4.6548, 14.3857] # glucose bounds
#     x = [0, 8.2358, 25.4840] # threonine bounds
    x = [0, 15.3220, 21.5844] # threonine bounds 2
    return (x[2]-x[1])/(y[2]-y[1])*(t-y[1]) + x[1]
#     return t - x[1] + y[1]
    
def update_uptake_bounds(model, t, met, coeffs, coeff_bounds):
    """Update the uptake rate of exchange reaction at timepoint t."""

    # Scale time trajectory for threonine
    if met in thr_mets:
        t = thr_time_transform(t)

    # Calculate logistic solutions with flexible number of parameters
    logistic_sol = logistic(t, *coeffs)
    uptake_rate = ddx_logistic(t, *coeffs[:-1])
    uptake_lb, uptake_ub, signal_lb, signal_ub = find_minmax_bounds(t, coeff_bounds)

    # Reverse direction for secretion reactions
    if met in ['glc', 'proL', 'leuL', 'thrL']:
        rid = 'Ex_' + met
        uptake_rate *= -1
        uptake_lb *= -1
        uptake_ub *= -1
        if met != 'leuL':
            model.reactions.get_by_id(rid).upper_bound = uptake_ub
    elif met in ['ival', 'isocap', '5av']:
        rid = 'Sec_' + met
        model.reactions.get_by_id(rid).upper_bound = uptake_ub
    elif met in ['2obut', '2hbut', '2abut', 'ppa']:
        rid = 'Pool_' + met
        model.reactions.get_by_id(rid).upper_bound = uptake_ub
    else:
        rid = 'Sec_' + met
    if met == 'leuL':
        model.reactions.get_by_id(rid).lower_bound = 0
        model.reactions.get_by_id(rid).upper_bound = 1000
    else:
        model.reactions.get_by_id(rid).lower_bound = uptake_lb
    return logistic_sol, uptake_rate, uptake_lb, uptake_ub, signal_lb, signal_ub


## Plot function

In [6]:
def areaplot(df, dl, du, t_max=48, ylabel='flux (mol/gDW/h)'):
    f = plt.figure(figsize=(14, 10), )#fontsize=30,
    ax = plt.axes(
        xticks=(range(0, t_max+1, 12)),
        xlim=(0, t_max),
        ylim=(0, 1.5),
    )
    for met, ser in df.iteritems():
        ser.index.name = 'index'
        p = ax.plot(ser.index.to_numpy(), ser.to_numpy(), '-', label=met, lw=5)
        color = matplotlib.colors.to_rgb(p[0].get_color())
        color = color + (0.2,)
        ax.fill_between(dl.index.to_numpy(), dl[met].to_numpy(), du[met].to_numpy(), color=color)
    ax.set_xlabel('time (h)', fontsize=30, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=30, fontweight='bold')
    plt.xticks(ax.get_xticks(), weight='bold')
    plt.yticks(ax.get_yticks(), weight='bold')
    
def niceplot(df, t_max=48, ylabel='flux (mol/gDW/h)'):
    ax = df.plot(
        figsize=(14, 10),
        xticks=(range(0, t_max+1, 12)),
        xlim=(0, t_max),
        ylim=(0, None),
        fontsize=30,
        lw=5,
    )
    ax.set_xlabel('time (h)', fontsize=30, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=30, fontweight='bold')
    plt.xticks(ax.get_xticks(), weight='bold')
    plt.yticks(ax.get_yticks(), weight='bold')

## Main dFBA program

In [7]:
def dfba_main(specsheet, traced, modelfile='../../data/icdf843-thr.json', t_max=48, resolution=1):
    """Main dFBA function. Computes successive FBA solutions and plots the concentration, uptake rates, and tracked reaction fluxes.
       
       Arguments:
       specsheet -- path to tsv sheet containing logistic parameters
       traced -- list of reaction ids to be traced over the timecourse
       
       Keyword arguments:
       modelfile -- location of metabolic model
       t_max -- end timepoint in hours (default 48)
       resolution -- number of hours per dFBA sample (default 1)
       """
    obj = 'ATP_sink'
    
    print(f"""dFBA log: Begin dFBA analysis with sheet
          {specsheet.split('/')[-1]}, endpoint
          {t_max} hours, and resolution {resolution}.""")
    
    # Load metabolic model and logistic fit specs #
    print('dFBA log: loading model and specsheet...')
    model = cb.io.load_json_model(modelfile)

    model.reactions.ID_508.lower_bound=0
    model.reactions.ID_508.upper_bound=0        
    model.reactions.ID_oxobutysynth.lower_bound=0
        
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    model.reactions.Ex_glc.upper_bound=0
    
    params = pd.read_excel(specsheet, sheet_name='cf', header=0, index_col=0)
    par_lb = pd.read_excel(specsheet, sheet_name='lb', header=0, index_col=0)
    par_ub = pd.read_excel(specsheet, sheet_name='ub', header=0, index_col=0)
    cf_count = params.count(axis=1) # count number of non-NA params for each metabolite
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    upta_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    upta_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    sign_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    sign_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnflux2 = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    print('dFBA log: simulation output begin.')
    mets = params.index.to_numpy()
    
    par_np = params.to_numpy()
    plb_np = par_lb.to_numpy()
    pub_np = par_ub.to_numpy()
    for i, t in enumerate(timecourse):
        for j, met in enumerate(mets):
            coeffs = [par_np[j, b] for b in range(cf_count[j])]
            coeff_bounds = [[plb_np[j, b], pub_np[j, b]] for b in range(cf_count[j])]
            logi, ddxlogi, ulb, uub, slb, sub = update_uptake_bounds(model, t, met, coeffs, coeff_bounds)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            upta_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = ulb
            upta_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = uub
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
            sign_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = slb
            sign_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = sub
        
        ## Get flux solution(s) and populate arrays ##
#         sol = model.optimize()
#         sol = lsol(model)
        sol = lsol(model)
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i+1}) \tFBA solution: {sol.fluxes[obj]}')
        if sol.fluxes[obj] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = sol.fluxes[rid]
        else:
            for rid in traced:
                rxnflux.at[t, model.reactions.get_by_id(rid).name] = sol.fluxes[rid]
        if i in [0, 6, 12, 24, 36, 48]:
            print(model.metabolites.alaL_c.summary(solution=sol))
            print("time = " + str(i))
            print(model.summary(solution=sol))
    
    # Adjust reaction directionality
    rids = [model.reactions.get_by_id("ID_383").name,
            model.reactions.get_by_id("ID_336").name,
            model.reactions.get_by_id("ID_391").name,
            model.reactions.get_by_id("ALDD3").name,
           ]
            
    for rid in rids:
        rxnflux[rid] *= -1             
    
    print(f'Complete after {i} cycles ({t} hours). Final flux: {sol.fluxes["Ex_biomass"]}')
    areaplot(uptakes, upta_lb, upta_ub, ylabel='flux (mM/h)')
    plt.show()
    niceplot(uptakes, ylabel='flux (mM/h)')
    plt.show()
#     plt.savefig('../../data/uptakes.png')
    niceplot(signals, ylabel='concentration (mM)')
    plt.show()
#     plt.savefig('../../data/concentrations.png')
    niceplot(rxnflux)
    plt.show()
#     plt.savefig('../../data/fluxes.png')
    rxnflux.to_csv('../../data/sumfluxes.csv', sep='\t')
#     rxnflux.to_excel('../../data/some_flux.xlsx', engine='xlsxwriter')
    writer = pd.ExcelWriter('../../data/uptakes.xlsx', engine='openpyxl')
    uptakes.to_excel(writer, sheet_name='data', engine='openpyxl')
    upta_lb.to_excel(writer, sheet_name='lbdev', engine='openpyxl')
    upta_ub.to_excel(writer, sheet_name='ubdev', engine='openpyxl')
    signals.to_excel(writer, sheet_name='signal', engine='openpyxl')
    sign_lb.to_excel(writer, sheet_name='slbdev', engine='openpyxl')
    sign_ub.to_excel(writer, sheet_name='subdev', engine='openpyxl')
    writer.save()

In [8]:
def dfva_main(specsheet, traced, modelfile='../../data/icdf843-thr.json', t_max=48, resolution=1):
    """Main dFBA function. Computes successive FBA solutions and plots the concentration, uptake rates, and tracked reaction fluxes.
       
       Arguments:
       specsheet -- path to tsv sheet containing logistic parameters
       traced -- list of reaction ids to be traced over the timecourse
       
       Keyword arguments:
       modelfile -- location of metabolic model
       t_max -- end timepoint in hours (default 48)
       resolution -- number of hours per dFBA sample (default 1)
       """
    
    obj = "ATP_sink"  # ATP_sink
    
    print(f"""dFVA log: Begin dFVA analysis with sheet
          {specsheet.split('/')[-1]}, endpoint
          {t_max} hours, and resolution {resolution}.""")
    
    # Load metabolic model and logistic fit specs #
    print('dFVA log: loading model and specsheet...')
    model = cb.io.load_json_model(modelfile)
    
    model.reactions.ID_508.lower_bound=0
    model.reactions.ID_508.upper_bound=0
    model.reactions.ID_oxobutysynth.lower_bound=0

    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03

    params = pd.read_excel(specsheet, sheet_name='cf', header=0, index_col=0)
    par_lb = pd.read_excel(specsheet, sheet_name='lb', header=0, index_col=0)
    par_ub = pd.read_excel(specsheet, sheet_name='ub', header=0, index_col=0)
    
    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    upta_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    uptakes = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    upta_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    sign_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), par_lb.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_lb.index])
    sign_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), par_ub.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in par_ub.index])
    signals = pd.DataFrame(np.zeros(shape=(len(timecourse), params.shape[0])), index=timecourse, columns=[model.metabolites.get_by_id(mid + '_c').name for mid in params.index])
    rxnflux = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced]) # 
    rxnf_lb = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    rxnf_ub = pd.DataFrame(np.zeros(shape=(len(timecourse), len(traced))), index=timecourse, columns=[model.reactions.get_by_id(rid).name for rid in traced])
    
    print('dFVA log: simulation output begin.')
    # Collect some input parameters
    mets = params.index.to_numpy()
    fulldata = [params.to_numpy(), par_lb.to_numpy(), par_ub.to_numpy()]

    # Iterate over time series, metabolites
    for i, t in enumerate(timecourse):
        for j, met in enumerate(mets):
            ## Assemble logistic coefficients ##
            data = [[a[j, b] for a in fulldata] for b in range(6)]
            ## Get flux constraints parameters and populate arrays ##
            logi, ddxlogi, ulb, uub, slb, sub = update_uptake_bounds(model, t, met, *data)
            uptakes.at[t, model.metabolites.get_by_id(met + '_c').name] = ddxlogi
            upta_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = ulb
            upta_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = uub
            signals.at[t, model.metabolites.get_by_id(met + '_c').name] = logi
            sign_lb.at[t, model.metabolites.get_by_id(met + '_c').name] = slb
            sign_ub.at[t, model.metabolites.get_by_id(met + '_c').name] = sub

        ## Get flux solution(s) and populate arrays ##        
        sol_b = lsol(model)
        for rid in traced:
            rxnflux.at[t, model.reactions.get_by_id(rid).name] = sol_b.fluxes[rid]        
        sol_v = fva(model, traced, fraction_of_optimum=0.995, loopless=True)
        for rid in traced:
            rxnf_ub.at[t, model.reactions.get_by_id(rid).name] = sol_v.at[rid,'maximum']
            rxnf_lb.at[t, model.reactions.get_by_id(rid).name] = sol_v.at[rid,'minimum']

        print(f'dFVA log: Time = {t}  (cycle {i+1}) \tFVA solution: {sol_b.fluxes[obj]}')

        if i in [0, 6, 12, 24, 36, 48]:
            print(model.summary(solution=sol_b))

    # Adjust reaction directionality
    rids = [model.reactions.get_by_id("ID_383").name,
            model.reactions.get_by_id("ID_336").name,
            model.reactions.get_by_id("ID_391").name,
            model.reactions.get_by_id("ALDD3").name,
           ]
            
    for rid in rids:
        rxnflux[rid] *= -1
        rxnf_ub[rid] *= -1
        rxnf_lb[rid] *= -1

    # Output plots and spreadsheets
    print(f'dFVA log: Complete after {i} cycles ({t} hours). Final flux: {sol_b.fluxes[obj]}')
    areaplot(uptakes, upta_lb, upta_ub, ylabel='flux (mM/h)')
    plt.show()
    niceplot(uptakes, ylabel='flux (mM/h)')
    plt.show()
#     plt.savefig('../../data/uptakes.png')
    niceplot(signals, ylabel='concentration (mM)')
    plt.show()
#     plt.savefig('../../data/concentrations.png')
    niceplot(rxnflux)
    plt.show()
#     plt.savefig('../../data/fluxes.png')
#     rxnflux.to_csv('../../data/sumfluxes.csv', sep='\t')
    writer = pd.ExcelWriter('../../data/fluxbounds.xlsx', engine='openpyxl')
    uptakes.to_excel(writer, sheet_name='data', engine='openpyxl')
    upta_lb.to_excel(writer, sheet_name='lbdev', engine='openpyxl')
    upta_ub.to_excel(writer, sheet_name='ubdev', engine='openpyxl')
    signals.to_excel(writer, sheet_name='signal', engine='openpyxl')
    sign_lb.to_excel(writer, sheet_name='slbdev', engine='openpyxl')
    sign_ub.to_excel(writer, sheet_name='subdev', engine='openpyxl')
    rxnflux.to_excel(writer, sheet_name='fluxes', engine='openpyxl')
    rxnf_ub.to_excel(writer, sheet_name='fluxub', engine='openpyxl')
    rxnf_lb.to_excel(writer, sheet_name='fluxlb', engine='openpyxl')
    writer.save()

## Run using glucose data

In [9]:
tracked_e = [
    "ID_391",
    "ID_53",
    "ID_36",
    "FHL2",
    "ID_280",
    "RNF-Complex",
    "ID_326",
    "ID_336",
    "ID_369",
    "ID_383",
    "BUK",
    "ID_660",
    "RXN-19534",
    "ICCoA-DHG-EB",
    "ID_314",
    "Pool_2abut",
    "Pool_2hbut",
    "ButSYNTH2",
    "PPAKr",
    "ALDD3",
    "ATP_sink",
]
tracked = [tracked_e[i-1] for i in [1, 2, 3, 5, 7, 6, 8, 13, 14, 15, 10, 11, 12, 16, 17, 18, 19, 20, 21]]
# dfba_main("../../data/coeffs.xlsx",
dfba_main("../../data/coeffs.xlsx", 
          tracked,
          modelfile='../../data/icdf843-thr.json',
          t_max=36,
          resolution=1,
         )

dFBA log: Begin dFBA analysis with sheet
          coeffs.xlsx, endpoint
          36 hours, and resolution 1.
dFBA log: loading model and specsheet...


KeyError: '2abut_c'

## Compute flux proportions for selected metabolites

In [182]:
def ac_prop(specsheet, met_id='acoa', modelfile="../../data/icdf843-thr.json", t_max=48, resolution=1, dry_run=True, switch=27):
    """Like dfba_main, but writes fluxes for acetate reactions.
    Plot with plotFluxArea.m
    """
    obj = 'ATP_sink'
        
    model = cb.io.load_json_model(modelfile)
    
    model.reactions.ID_508.lower_bound=0
    model.reactions.ID_508.upper_bound=0
    model.reactions.ID_oxobutysynth.lower_bound=0
    
    init_cnc = dict()
    for rxn in model.reactions:
        if rxn.id.startswith('Ex_') and rxn.id.endswith('L') or rxn.id in ['Ex_gly', 'Ex_his']:
            init_cnc[rxn.id] = rxn.upper_bound
            rxn.upper_bound *= 0.03
    model.reactions.Ex_glc.upper_bound=0
    
    params = pd.read_excel(specsheet, sheet_name='cf', header=0, index_col=0)
    par_lb = pd.read_excel(specsheet, sheet_name='lb', header=0, index_col=0)
    par_ub = pd.read_excel(specsheet, sheet_name='ub', header=0, index_col=0)

    # Initialize results dataframes
    timecourse = list(range(0, t_max + resolution, resolution))
    
    ac_rxns_in = []
    ac_data_in = []
    ac_rxns_ot = []
    ac_data_ot = []
    
    try:
        ac = model.metabolites.get_by_id(met_id + "_c")
    except KeyError:
        print(f"'{met_id}_c' is not a valid metabolite.")
        print('END')
        return
    
    print('dFBA log: simulation output begin.')
    mets = params.index.to_numpy()
    fulldata = [params.to_numpy(), par_lb.to_numpy(), par_ub.to_numpy()]
    # Iterate over time series, metabolites
    for i, t in enumerate(timecourse):
        for j, met in enumerate(mets):
            ## Assemble logistic coefficients ##
            data = [[a[j, b] for a in fulldata] for b in range(6)]
            update_uptake_bounds(model, t, met, *data)
        sol = lsol(model)
        if i % 10 == 0:  
            print(f'dFBA log: Time = {t}  (cycle {i}) \tFBA solution: {sol.fluxes["ATP_sink"]}')
        if sol.fluxes[obj] < 0.0001:
            print(f'dFBA log: infeasible solution on cycle {i}.')
        if i in [0, 12, 20, 24, 36, 48]:
            print("time = " + str(i))
            print(ac.summary(solution=sol))

        localdata_in = {}
        localdata_ot = {}
        for rxn in model.reactions:
            if (ac in rxn.reactants and sol.fluxes[rxn.id] > 1E-5) or (ac in rxn.products and sol.fluxes[rxn.id] < -1E-5):
                localdata_ot[rxn.id] = rxn.metabolites[ac] * sol.fluxes[rxn.id]
            if (ac in rxn.reactants and sol.fluxes[rxn.id] < -1E-5) or (ac in rxn.products and sol.fluxes[rxn.id] > 1E-5):
                localdata_in[rxn.id] = rxn.metabolites[ac] * sol.fluxes[rxn.id]
        ac_data_in.append(localdata_in)
        ac_data_ot.append(localdata_ot)
        for k in localdata_in:
            ac_rxns_in.extend([k for k in localdata_in])
        for k in localdata_ot:
            ac_rxns_ot.extend([k for k in localdata_ot])
            
    if not dry_run:    
        rids_in = list(set(ac_rxns_in))
        rids_ot = list(set(ac_rxns_ot))
        flxs_in = pd.DataFrame(data=np.zeros((len(timecourse), len(rids_in))), index=timecourse, columns=rids_in)
        flxs_ot = pd.DataFrame(data=np.zeros((len(timecourse), len(rids_ot))), index=timecourse, columns=rids_ot)
        rnms_in = [model.reactions.get_by_id(rid).name for rid in rids_in]
        rnms_ot = [model.reactions.get_by_id(rid).name for rid in rids_ot]
        with open(f'../../data/{met_id}flux_in.txt', 'w') as wfi, open(f'../../data/{met_id}flux_ot.txt', 'w') as wfo:
            wfi.write('time\t' + '\t'.join(rids_in) + '\n')
            wfi.write('time\t' + '\t'.join(rnms_in) + '\n')
            wfo.write('time\t' + '\t'.join(rids_ot) + '\n')
            wfo.write('time\t' + '\t'.join(rnms_ot) + '\n')
            for i, t in enumerate(timecourse):
                wfi.write(str(t))
                wfo.write(str(t))
                for rxn in rids_in:
                    val = ac_data_in[i].get(rxn, 0)
                    wfi.write('\t' + str(val))
                    flxs_in.at[t, rxn] = val
                for rxn in rids_ot:
                    val = ac_data_ot[i].get(rxn, 0)
                    wfo.write('\t' + str(val))
                    flxs_ot.at[t, rxn] = val
                wfi.write('\n')
                wfo.write('\n')
        print("Produced flux percentages:")
        totalflux = flxs_in.to_numpy().sum()
        cuts = flxs_in.sum(axis=0)
        for i, rxn in enumerate(rids_in):
            print(rxn, rnms_in[i], f"{cuts.at[rxn]/totalflux*100:.2f}")
        print("Consumed flux percentages:")
        totalflux = flxs_ot.to_numpy().sum()
        cuts = flxs_ot.sum(axis=0)
        for i, rxn in enumerate(rids_ot):
            print(rxn, rnms_ot[i], f"{cuts.at[rxn]/totalflux*100:.2f}")
        print("Absolute influx:")        
        cuts = flxs_in.sum(axis=0)
        for i, rxn in enumerate(rids_in):
            print(rxn, rnms_in[i], f"{cuts.at[rxn]}")
        print("Absolute outflux:")
        cuts = flxs_ot.sum(axis=0)
        for i, rxn in enumerate(rids_ot):
            print(rxn, rnms_ot[i], f"{cuts.at[rxn]}")
        

In [183]:
ac_prop("../../data/coeffs.xlsx", 
          'gluL',
          t_max=36,
          resolution=1,
          dry_run=False,
          switch=0,
         )

dFBA log: simulation output begin.
dFBA log: Time = 0  (cycle 0) 	FBA solution: 2.2667669960126466
time = 0
gluL_c
Formula: C5H9NO4

Producing Reactions
-------------------
Percent    Flux  Reaction                              Definition
 24.44% 0.05085 ALT_2abut 2obut_c + gluL_c <=> 2abut_c + 2oglut_c
 14.12% 0.02937    ID_217   2oglut_c + valL_c --> 2oiv_c + gluL_c
 61.44%  0.1278     ID_28  4m2op_c + gluL_c <=> 2oglut_c + leuL_c

Consuming Reactions
-------------------
Percent   Flux Reaction                                                 Definition
100.00% -0.208   ID_575 gluL_c + h2o_c + nad_c --> 2oglut_c + h_c + nadh_c + nh3_c
dFBA log: Time = 10  (cycle 10) 	FBA solution: 4.489088053859987
time = 12
gluL_c
Formula: C5H9NO4

Producing Reactions
-------------------
Percent   Flux  Reaction                              Definition
 27.63% 0.1604 ALT_2abut 2obut_c + gluL_c <=> 2abut_c + 2oglut_c
 72.37% 0.4201     ID_28  4m2op_c + gluL_c <=> 2oglut_c + leuL_c

Consuming Reactions


In [124]:
model = cb.io.load_json_model('../../data/icdf843-thr.json')
for rxn in model.metabolites.get_by_id('ppa_c').reactions:
    print(rxn.id, rxn.name, rxn.reaction)
print(model.reactions.ID_508.reaction)

PPAtex Propionate transport via diffusion ppa_c <=> ppa_e
Pool_ppa Pooling Propionate ppa_c --> 
PPAKr Propionate kinase adp_c + ppap_c <=> atp_c + ppa_c
ALDD3 Aldehyde dehydrogenase propanal h2o_c + nad_c + ppal_c <=> 2 h_c + nadh_c + ppa_c
pyam_c + pyr_c <=> alaL_c + pyal_c
